In [ ]:
import tensorflow as tf
global_step = tf.Variable(0)
# 通过eponentiial_decay函数生成学习率
learning_rate = tf.train.exponential_decay(0.1, global_step,100,0.96,staircase = True)
#使用指数衰减的学习率，在minimize函数中传入global_step 将自动更新
learning_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)

In [ ]:
#学习率的设置
w = tf.Variable(tf.random_normal([2,1],stddev=1,seed =1))
y= tf.matmul(x,w)

loss =tf.reduce_mean(tf.square(y_ -y)) + tf.contrib.layers.l2_regularizer(lambda)(w)

In [8]:
# 正则化防止过拟合
weights = tf.constant([[1.,-2.],[-3.,4.]])
with tf.Session() as sess:
    print(sess.run(tf.contrib.layers.l1_regularizer(0.5)(weights)))

with tf.Session() as sess:
    print(sess.run(tf.contrib.layers.l2_regularizer(0.5)(weights)))


5.0
7.5


In [36]:

# 5层的L2正则化的损失函数
import tensorflow as tf
from numpy.random import RandomState
import numpy as np
#获取一层神经网络边上的权重，并将这个权重的L2正则化损失加入名称为'losses'的集合中
def get_weight(shape,l2_weight):
#生成一个变量
    var = tf.Variable(tf.random_normal(shape),dtype=tf.float32)
    # add_to_collection 函数将这个新生成变量的L2正则化损失项加入集合
    tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(l2_weight)(var))
    return var

tf.reset_default_graph()
x = tf.placeholder(tf.float32,shape=(None,2))
y_ = tf.placeholder(tf.float32,shape=(None,1))

#生成模拟数据
rdm = RandomState(1)
dataset_size =128
X = rdm.rand(dataset_size,2)
# Y = [[int(x1+x2<1)] for (x1,x2) in X]
Y = np.random.randint(0,2,128)
Y =[[x] for x in Y]
batch_size = 8

# 定义了每一层网络节点的个数
layer_dimension = [2,10,10,10,1]

#神经网络的层数
n_layers = len(layer_dimension)

# 这个变量维护向前传播时最深层的节点，开始的时候就是输入层
cur_layer = x
# 当前层的节点个数
in_dimension =layer_dimension[0]

#通过一个循环来生成5层的全连接的神经网络结构
for i in range(1,n_layers):
    #lay_dimension[i]为下一层的节点数
    out_dimension = layer_dimension[i]
    #生成当前层中权重的变量，并将这个变量的L2正则化加入计算图上的集合
    weight = get_weight([in_dimension,out_dimension],0.001)
    bias = tf.Variable(tf.constant(0.1,shape=[out_dimension]))
    cur_layer = tf.nn.relu(tf.matmul(cur_layer,weight)+bias)
    in_dimension = out_dimension
    
# 在定义神经网络前向传播的同时已经将所有的L2正则化损失加入到图上的集合
#这里只需要计算刻画模型在训练数据上表现的损失函数
mse_loss =tf.reduce_mean(tf.square(y_ -cur_layer))
# 将均方误差函数加入到损失函数
tf.add_to_collection('losses',mse_loss)
#get_collection返回一个列表，这个列表是所有这个集合中的元素，这些元素是损失函数的不同部分，将它们加起来可以得到最终的损失函数
loss = tf.add_n(tf.get_collection('losses'))
train_step = tf.train.AdamOptimizer(0.01).minimize(loss)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    STEPS = 100
    for i in range(STEPS):
        start = (i*batch_size)%dataset_size
        end = min(start+batch_size,dataset_size)
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
        print(sess.run(loss,feed_dict={x:X[start:end],y_:Y[start:end]}))
#         print(loss.eval())


In [36]:
#滑动平均模型
import tensorflow as tf
# 定义一个变量用于计算滑动平均
v1 = tf.Variable(0,dtype = tf.float32)
# 模拟神经网络中迭代的次数，可以用于动态控制衰减率
step = tf.Variable(0,trainable=False)
# 定义一个滑动平均类，初始化时给了衰减率（0.99）和控制衰减率的变量step
ema = tf.train.ExponentialMovingAverage(0.99,step)
# 定义一个更新变量滑动平均的操作，这里是一个列表，每执行一次操作，这个列表中的变量都会更新
maintain_average_op = ema.apply([v1])

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run([v1,ema.average(v1)]))
#     更新变量v1的值到5
    sess.run(tf.assign(v1,5))
    sess.run(maintain_average_op)
    
    print(sess.run([v1,ema.average(v1)]))
    
    sess.run(tf.assign(step,10000))
    sess.run(tf.assign(v1,10))
    sess.run(maintain_average_op)
    print(sess.run([v1,ema.average(v1)]))
    
    sess.run(maintain_average_op)
    print(sess.run([v1,ema.average(v1)]))


[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.555]
[10.0, 4.60945]


In [27]:
import tensorflow as tf
from numpy.random import RandomState
import numpy as np
#获取一层神经网络边上的权重，并将这个权重的L2正则化损失加入名称为'losses'的集合中
def get_weight(shape,lambda1):
    #生成一个变量 
    var=tf.Variable(tf.random_normal(shape,stddev=1,seed=1))
    #add_to_collection 函数将这个新生成变量的L2正则化损失项加入集合
    #这个函数第一个参数losses是集合的名字，第二个参数是要加入这个集合的内容
    tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(lambda1)(var))
    return var

batch_size=8

#两个输入节点,回归问题一般只有
x=tf.placeholder(tf.float32,shape=(None,2),name="x-input")
y_=tf.placeholder(tf.float32,shape=(None,1),name="y-input")

#通过随机数生成一个模拟数据集
rdm=RandomState(1)
dataset_size=128
X = rdm.rand(dataset_size,2)
# X=np.array(rdm.rand(dataset_size,2),dtype=np.float32)

#设置回归的正确值为两个输入的和加上一个随机量。加上随机量是为了加入不可预测的噪音，否则不同损失函数的意义就不大了
#因为不同损失函数都会在能完全预测正确的时候最低。一般来说，噪音为一个均值为0的小量，所以这里噪声设置为-0.05~0.05的随机数
Y=[[int(x1+x2<1)] for (x1,x2) in X]
# Y =np.array(Y,dtype=np.float32)

layer_dimension=[2,10,5,3,1]
n_layers=len(layer_dimension)

#这个变量维护前向传播时最深层的节点，开始的时候就是输入层
cur_layer=x
# 当前层的节点个数
in_dimension=layer_dimension[0]

#通过一个循环来生成5层全连接的神经网络结构
for i in range(1,n_layers):
    # layer_demension[i]为下一层的节点数
    out_dimension=layer_dimension[i]
    #生成当前层中权重的变量，并将这个变量的L2正则化损失加入计算图上的集合
    weight=get_weight([in_dimension,out_dimension],0.003)
    bias=tf.Variable(tf.constant(0.1,shape=[out_dimension]))
    #使用Relu激活函数
    cur_layer=tf.nn.relu(tf.matmul(cur_layer,weight)+bias)
    #进入下一层之前将下一层的节点个数更新为当前节点个数
    in_dimension=layer_dimension[i]

#在定义神经网络前向传播的同时已经将所有的L2正则化损失加入了图上的集合，
#这里只需要计算刻画模型在训练数据上表现的损失函数
mse_loss=tf.reduce_mean(tf.square(y_-cur_layer))

#将均方误差损失函数加入损失集合
tf.add_to_collection('losses',mse_loss)

#get_collection返回一个列表，这个列表是所有这个集合中的元素。在这个样例中，
#这些元素就是损失函数的不同部分，将它们加起来就可以得到最终的损失函数
loss=tf.add_n(tf.get_collection('losses'))
train_step=tf.train.AdamOptimizer(0.001).minimize(mse_loss)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(5000):
        start=(i*batch_size)%dataset_size
        end=min(start+batch_size,dataset_size)
#         print('******')
#         print(X[start:end],'\n',Y[start:end])
#         print('******')
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
        print(sess.run(loss))

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float and shape [?,1]
	 [[Node: Placeholder_1 = Placeholder[dtype=DT_FLOAT, shape=[?,1], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_1', defined at:
  File "d:\python3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\python3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\python3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\python3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "d:\python3\lib\site-packages\ipykernel\kernelapp.py", line 478, in start
    self.io_loop.start()
  File "d:\python3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "d:\python3\lib\site-packages\tornado\ioloop.py", line 888, in start
    handle

In [23]:
import tensorflow as tf
from numpy.random import RandomState

batch_size=8

#两个输入节点,回归问题一般只有
x=tf.placeholder(tf.float32,shape=(None,2),name="x-input")
y_=tf.placeholder(tf.float32,shape=(None,1),name="y-input")
#定义一个单层的神经网络前向传播的过程，这里是简单的加权和
w1=tf.Variable(tf.random_normal([2,1],stddev=1,seed=1))
y=tf.matmul(x,w1)
#定义预测多了和预测少了的成本
loss_less=10
loss_more=1
loss=tf.reduce_sum(tf.where(tf.greater(y,y_),(y-y_)*loss_more,(y_-y)*loss_less))
train_step=tf.train.AdamOptimizer(0.001).minimize(loss) #优化器

#通过随机数生成一个模拟数据集
rdm=RandomState(1)
dataset_size=128
X=rdm.rand(dataset_size,2)

#设置回归的正确值为两个输入的和加上一个随机量。加上随机量是为了加入不可预测的噪音，否则不同损失函数的意义就不大了
#因为不同损失函数都会在能完全预测正确的时候最低。一般来说，噪音为一个均值为0的小量，所以这里噪声设置为-0.05~0.05的随机数
Y=[[x1+x2+rdm.rand()/10.0-0.05] for (x1,x2) in X]

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(500):
        start=(i*batch_size)%dataset_size
        end=min(start+batch_size,dataset_size)
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
        print(sess.run(w1))

[[-0.81031823]
 [ 1.4855988 ]]
[[-0.8093366]
 [ 1.4865911]]
[[-0.80838311]
 [ 1.48758912]]
[[-0.80742168]
 [ 1.48854196]]
[[-0.80644417]
 [ 1.4894979 ]]
[[-0.80545688]
 [ 1.49047196]]
[[-0.80446237]
 [ 1.49145281]]
[[-0.8034634 ]
 [ 1.49243844]]
[[-0.80245554]
 [ 1.49342489]]
[[-0.80146426]
 [ 1.49441588]]
[[-0.80048096]
 [ 1.49542165]]
[[-0.79950386]
 [ 1.4964081 ]]
[[-0.79850495]
 [ 1.49741769]]
[[-0.79752892]
 [ 1.49840248]]
[[-0.79654419]
 [ 1.4994123 ]]
[[-0.79556257]
 [ 1.50043571]]
[[-0.79462433]
 [ 1.50143182]]
[[-0.79369748]
 [ 1.50242686]]
[[-0.79274493]
 [ 1.50342429]]
[[-0.79180086]
 [ 1.50438201]]
[[-0.79083252]
 [ 1.50532436]]
[[-0.78985554]
 [ 1.50629151]]
[[-0.78887033]
 [ 1.50725818]]
[[-0.78787929]
 [ 1.50822437]]
[[-0.78687394]
 [ 1.50918674]]
[[-0.78588563]
 [ 1.51015079]]
[[-0.78490686]
 [ 1.51113391]]
[[-0.7839362 ]
 [ 1.51209509]]
[[-0.78293514]
 [ 1.51309192]]
[[-0.7819587 ]
 [ 1.51406181]]
[[-0.78097296]
 [ 1.51506674]]
[[-0.77999133]
 [ 1.51609015]]
[[-0.77905

[[-0.45050699]
 [ 1.81899869]]
[[-0.44964769]
 [ 1.81980479]]
[[-0.44880766]
 [ 1.82061625]]
[[-0.44792995]
 [ 1.82143486]]
[[-0.44705686]
 [ 1.82222831]]
[[-0.44613892]
 [ 1.82302451]]
[[-0.44522291]
 [ 1.82381594]]
[[-0.44428566]
 [ 1.82463503]]
[[-0.44333118]
 [ 1.82547855]]
[[-0.44235182]
 [ 1.82632375]]
[[-0.44141069]
 [ 1.82711172]]
[[-0.44049159]
 [ 1.82790339]]
[[-0.43958265]
 [ 1.82866406]]
[[-0.43861714]
 [ 1.82951653]]
[[-0.43768802]
 [ 1.83032227]]
[[-0.43676132]
 [ 1.8311516 ]]
[[-0.43585163]
 [ 1.83197606]]
[[-0.43499279]
 [ 1.83278239]]
[[-0.43415305]
 [ 1.83359456]]
[[-0.43327537]
 [ 1.83441424]]
[[-0.43240219]
 [ 1.83520889]]
[[-0.43148389]
 [ 1.83600676]]
[[-0.43056744]
 [ 1.8368001 ]]
[[-0.42962956]
 [ 1.83762145]]
[[-0.42867431]
 [ 1.8384676 ]]
[[-0.42769402]
 [ 1.83931565]]
[[-0.426752  ]
 [ 1.84010637]]
[[-0.42583197]
 [ 1.8409009 ]]
[[-0.42493862]
 [ 1.84162879]]
[[-0.42398676]
 [ 1.84245288]]
[[-0.42306983]
 [ 1.84323323]]
[[-0.42215395]
 [ 1.84404027]]
[[-0.421